# How to use a cohort

This notebook shows how to use a cohort saved from Data Explorer.

It uses a cohort saved in the [Terra Notebooks Playground workspace](https://app.terra.bio/#workspaces/help-gatk/Terra%20Notebooks%20Playground/data).

## Setup

First, be sure to run notebook **`Python environment setup`** in this workspace.

Then in this section we:

1. load the needed python packages
2. set the project id of the cloud project to bill for queries to BigQuery

In [4]:
import os

import pandas as pd
import firecloud.api as fapi

In [7]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']

## Retrieve cohort SQL query

In [8]:
# Hard-code instead of use WORKSPACE_NAMESPACE/WORKSPACE_NAME, since other workspaces
# won't have the 1000g_americans cohort.
ws_namespace = 'help-gatk'
ws_name = 'Terra Notebooks Playground'
cohort_query = fapi.get_entity(ws_namespace, ws_name, 'cohort', '1000g_americans').json()['attributes']['query']
cohort_query

'SELECT DISTINCT t1.participant_id FROM (SELECT participant_id FROM `verily-public-data.human_genome_variants.1000_genomes_participant_info` WHERE  ((Super_Population_Description = "American"))) t1'

## Create pandas dataframe of cohort participant ids

In [9]:
participant_ids = pd.read_gbq(
    cohort_query,
    dialect='standard')
participant_ids.head()

,participant_id
0,HG01433
1,HG01445
2,HG01452
3,HG01473
4,HG01482


## See what tables are available to join against

In [12]:
bq_table_entities = fapi.get_entities(ws_namespace, ws_name, 'BigQuery_table').json()
bq_tables = list(map(lambda e: e['attributes']['table_name'], bq_table_entities))
bq_tables

['verily-public-data.human_genome_variants.1000_genomes_participant_info',
 'verily-public-data.human_genome_variants.1000_genomes_sample_info']

## Join cohort participant ids against sample_info table

In [13]:
sample_info = pd.read_gbq("SELECT * FROM `verily-public-data.human_genome_variants.1000_genomes_sample_info`",
                          dialect="standard")
print("sample_info has %d rows" % len(sample_info.index))

sample_info_americans = participant_ids.join(sample_info, lsuffix='_L', rsuffix='_R')
print("sample_info_americans has %d rows\n" % len(sample_info_americans.index))

sample_info_americans.head()

sample_info has 3500 rows
sample_info_americans has 1092 rows



,participant_id_L,sample_id,participant_id_R,In_Low_Coverage_Pilot,LC_Pilot_Platforms,LC_Pilot_Centers,In_High_Coverage_Pilot,HC_Pilot_Platforms,HC_Pilot_Centers,In_Exon_Targetted_Pilot,...,exome_mapped_cram,exome_mapped_crai,exome_mapped_csra,exome_unmapped_bam,exome_unmapped_bai,exome_unmapped_bas,wgs_high_cov_bam,wgs_high_cov_bai,wgs_high_cov_cram,wgs_high_cov_crai
0,HG01433,HG00144,HG00144,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,HG01433,NA20287,NA20287,None,None,None,None,None,None,None,...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,None,None,None,None
0,HG01433,NA11994,NA11994,True,"ABI_SOLID,ILLUMINA","BCM,BI",None,None,None,True,...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,None,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,None,None,None,None
1,HG01445,HG00147,HG00147,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,HG01445,NA20289,NA20289,None,None,None,None,None,None,None,...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,gs://genomics-public-data/ftp-trace.ncbi.nih.g...,None,None,None,None


# Provenance

In [ ]:
import datetime
print(datetime.datetime.now())
!pip3 freeze

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.